## nfl lines machine learning project


source data for historical spreads and game scores
http://www.repole.com/sun4cast/data.html

In [1]:
import os

# define the root directory for the nfl code in $MLNLF_ROOT
codeDir = "".join([os.environ['MLNFL_ROOT'], os.path.sep])
dataRoot = "".join([codeDir, "data", os.path.sep])

os.chdir(codeDir)

print(codeDir)
print(dataRoot)

/Users/amit/repos/mlnfl/nfl/
/Users/amit/repos/mlnfl/nfl/data/


In [2]:
# warnings control
import warnings
# choose default, ignore, always
warnings.filterwarnings('default')


In [3]:
# import necessary modules
%matplotlib inline

from __future__ import division
from __future__ import print_function
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import madden

from sklearn import linear_model
from sklearn import svm
from sklearn import ensemble

print ("pandas version ",pd.__version__)

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/pylab/config.py:95: DeprecationW

pandas version  0.18.0


In [4]:
# location of lookup files
from referencedata import ReferenceData
lookupFiles = { 'teams' : {'file': 'nflTeams.csv' },
                'seasons' : {'file': 'seasons.csv' },
               }

lookupDir = "".join([dataRoot, 'lookup', os.path.sep])

print ("lookupFiles = %s" % lookupFiles)
print ("lookupDir = %s" % lookupDir)

lookupFiles = {'seasons': {'file': 'seasons.csv'}, 'teams': {'file': 'nflTeams.csv'}}
lookupDir = /Users/amit/repos/mlnfl/nfl/data/lookup/


In [5]:
# import reference data
reference_data = ReferenceData(lookupDir)
reference_data.teams_df.head()

,city,mascot,league,division,year
team,,,,,
Baltimore Ravens,NaN,Ravens,afc,north,NaN
New England Patriots,NaN,Patriots,afc,east,NaN
Tennessee Titans,NaN,Titans,afc,south,NaN
Atlanta Falcons,NaN,Falcons,nfc,south,NaN
Tampa Bay Buccaneers,NaN,Buccaneers,nfc,south,NaN


In [13]:
###  multi-season training
reload(madden)

# testYears and trainYears need to be arrays
trainYears = [2014] #range(2008,2013) 
testYear = [2015]

# train on previous 3 yrs of data
testYear = [2016]
trainYears = range(testYear[0]-3,testYear[0]) 

# training data set - includes one extra year for prev yr record
seasons = np.array(trainYears)
print ("training seasons >> ", seasons)

training seasons >>  [2013 2014 2015]


In [14]:
reload(madden)

# get training data
# 1 - read all the games
path_to_lines = dataRoot + "lines/"
dfAllGames = madden.readGamesAll(path_to_lines, seasons)
# 2 - compile season record for all teams
dfAllTeams = madden.seasonRecord(dfAllGames, reference_data)
# 3 - apply season records and compute other fields for all games
dfAllGames = madden.processGames(dfAllGames, dfAllTeams, reference_data)
# 4 - remove extra year of data 
dfAllGames = dfAllGames[dfAllGames.season.isin(seasons)]

# use different test set 
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)
# 1 - read all the games
dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)  
# 2 - compile season record for all teams
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data) 
# 3 - apply season records and compute other fields for all games
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data) 
# 4 - remove extra year of data 
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]


results for >>  [2016]


In [15]:
dfAllGames

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,week,favorite,...,homeWin,favoredWin,gameWeek,homeRecord,visitorRecord,favoredRecord,underdogRecord,prevFavoredRecord,prevUnderdogRecord,absLine
8174,9/5/13,Baltimore Ravens,27.0,Denver Broncos,49.0,7.5,49.5,2013,NaN,Denver Broncos,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.8125,0.6250,7.5
8175,9/8/13,New England Patriots,23.0,Buffalo Bills,21.0,-10.5,51.5,2013,NaN,New England Patriots,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.7500,0.3750,10.5
8176,9/8/13,Tennessee Titans,16.0,Pittsburgh Steelers,9.0,6.0,42,2013,NaN,Pittsburgh Steelers,...,0,0,1,0.000000,0.000000,0.000000,0.000000,0.5000,0.3750,6.0
8177,9/8/13,Atlanta Falcons,17.0,New Orleans Saints,23.0,3.5,56,2013,NaN,New Orleans Saints,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.4375,0.8125,3.5
8178,9/8/13,Tampa Bay Buccaneers,17.0,New York Jets,18.0,-6.0,39,2013,NaN,Tampa Bay Buccaneers,...,1,0,1,0.000000,0.000000,0.000000,0.000000,0.4375,0.3750,6.0
8179,9/8/13,Kansas City Chiefs,28.0,Jacksonville Jaguars,2.0,-4.5,43,2013,NaN,Kansas City Chiefs,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.1250,0.1250,4.5
8180,9/8/13,Cincinnati Bengals,21.0,Chicago Bears,24.0,3.0,42,2013,NaN,Chicago Bears,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.6250,0.6250,3.0
8181,9/8/13,Miami Dolphins,23.0,Cleveland Browns,10.0,2.5,40.5,2013,NaN,Cleveland Browns,...,0,0,1,0.000000,0.000000,0.000000,0.000000,0.3125,0.4375,2.5
8182,9/8/13,Seattle Seahawks,12.0,Carolina Panthers,7.0,-3.5,45,2013,NaN,Seattle Seahawks,...,0,1,1,0.000000,0.000000,0.000000,0.000000,0.6875,0.4375,3.5
8183,9/8/13,Minnesota Vikings,24.0,Detroit Lions,34.0,4.5,47.5,2013,NaN,Detroit Lions,...,1,1,1,0.000000,0.000000,0.000000,0.000000,0.2500,0.6250,4.5


In [16]:
# define independent variables for logistic regression
features = ['favoredRecord','underdogRecord',  # current year records of both teams
            'prevFavoredRecord','prevUnderdogRecord', # prev year records, helps early in season when only few games played
            'gameWeek',  # week in season, should make a good/bad record later in season more important
            'absLine',  # absolute value of spread since favored team already determined
            'divisionGame', # T/F, usually more competitive rivalry games, i.e. bad teams still win home division games.
            'favoredHomeGame', # T/F, important since output of classifier is "did the favored team win?"
            ]

# run the classifer
random_state = 11
#classifier = svm.SVC(kernel='poly',probability=True, random_state=random_state)
classifier = linear_model.LogisticRegression(C=1e5)
mlClassifier = madden.runScikitClassifier(dfAllGames,features,classifier)

# apply results of logistic regression to the test set
#dfPredict = madden.predictGames(dfGamesTest,mlClassifier,features)

# apply ranking logic and determine scoring outcomes for league
#dfAll = madden.rankGames(dfPredict,reference_data,seasonTest[0])

In [17]:
# predict one week of current season
iweek = 9
reload(madden)

# use different test set - current year
testYear = [2016]
seasonTest = np.array(testYear) # should be only one year
print ("results for >> ", seasonTest)

dfGamesTest = madden.readGamesAll(path_to_lines, seasonTest)
dfTeamsTest = madden.seasonRecord(dfGamesTest,reference_data)
dfGamesTest = madden.processGames(dfGamesTest, dfTeamsTest, reference_data)
dfGamesTest = dfGamesTest[dfGamesTest.season.isin(seasonTest)]

# pick only this weeks games for predict
dfTest = dfGamesTest[dfGamesTest.gameWeek == iweek]

# apply results of logistic regression to the test set
dfPredict = madden.predictGames(dfTest,mlClassifier,features)
# apply ranking logic and determine scoring outcomes for league
dfAll = madden.rankGames(dfPredict, reference_data, seasonTest[0])



results for >>  [2016]


In [18]:
dfGamesTest[dfGamesTest.gameWeek == iweek]

,Date,Visitor,Visitor Score,Home Team,Home Score,Line,Total Line,season,week,favorite,...,homeWin,favoredWin,gameWeek,homeRecord,visitorRecord,favoredRecord,underdogRecord,prevFavoredRecord,prevUnderdogRecord,absLine
9062,11/3/16,Atlanta Falcons,NaN,Tampa Bay Buccaneers,NaN,-3.5,NaN,2016,9.0,Atlanta Falcons,...,NaN,NaN,9,0.375,0.625,0.625,0.375,0.5000,0.3750,3.5
9063,11/6/16,Dallas Cowboys,NaN,Cleveland Browns,NaN,-7.5,NaN,2016,9.0,Dallas Cowboys,...,NaN,NaN,9,0.000,0.750,0.750,0.000,0.2500,0.1875,7.5
9064,11/6/16,Detroit Lions,NaN,Minnesota Vikings,NaN,6.0,NaN,2016,9.0,Minnesota Vikings,...,NaN,NaN,9,0.625,0.500,0.625,0.500,0.6875,0.4375,6.0
9065,11/6/16,Jacksonville Jaguars,NaN,Kansas City Chiefs,NaN,7.5,NaN,2016,9.0,Kansas City Chiefs,...,NaN,NaN,9,0.625,0.250,0.625,0.250,0.6875,0.3125,7.5
9066,11/6/16,New York Jets,NaN,Miami Dolphins,NaN,3.5,NaN,2016,9.0,Miami Dolphins,...,NaN,NaN,9,0.375,0.375,0.375,0.375,0.3750,0.6250,3.5
9067,11/6/16,Philadelphia Eagles,NaN,New York Giants,NaN,2.5,NaN,2016,9.0,New York Giants,...,NaN,NaN,9,0.500,0.500,0.500,0.500,0.3750,0.4375,2.5
9068,11/6/16,Pittsburgh Steelers,NaN,Baltimore Ravens,NaN,0.1,NaN,2016,9.0,Baltimore Ravens,...,NaN,NaN,9,0.375,0.500,0.375,0.500,0.3125,0.6250,0.1
9069,11/6/16,Carolina Panthers,NaN,Los Angeles Rams,NaN,-3.0,NaN,2016,9.0,Carolina Panthers,...,NaN,NaN,9,0.375,0.250,0.250,0.375,0.9375,0.4375,3.0
9070,11/6/16,New Orleans Saints,NaN,San Francisco 49ers,NaN,-3.5,NaN,2016,9.0,New Orleans Saints,...,NaN,NaN,9,0.125,0.375,0.375,0.125,0.4375,0.3125,3.5
9071,11/6/16,Indianapolis Colts,NaN,Green Bay Packers,NaN,7.0,NaN,2016,9.0,Green Bay Packers,...,NaN,NaN,9,0.500,0.375,0.500,0.375,0.6250,0.5000,7.0


In [20]:
# display weekly ranking output

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

dispCols = ['season','gameWeek','Visitor','visitorRecord','Home Team','homeRecord',
            'Line','prevFavoredRecord','prevUnderdogRecord','predict_proba',
            'lineGuess','probaGuess', 'probaAbsGuess', 'predictTeam']


dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
guessCol = 'probaGuess'
predictCols = ['gameWeek','predictTeam', 'predict_proba', guessCol, 'favorite','lineGuess', 'Line']
dfAll[predictCols].sort(guessCol, ascending=False)

/Users/amit/.virtualenvs/datascience/lib/python2.7/site-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,gameWeek,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
9065,9,Kansas City Chiefs,0.815774,16.0,Kansas City Chiefs,16.0,7.5
9063,9,Dallas Cowboys,0.800223,15.0,Dallas Cowboys,15.0,-7.5
9064,9,Minnesota Vikings,0.729041,14.0,Minnesota Vikings,12.0,6.0
9071,9,Green Bay Packers,0.722062,13.0,Green Bay Packers,13.0,7.0
9074,9,Seattle Seahawks,0.685294,12.0,Seattle Seahawks,14.0,7.0
9062,9,Atlanta Falcons,0.615777,11.0,Atlanta Falcons,9.0,-3.5
9070,9,New Orleans Saints,0.594860,10.0,New Orleans Saints,8.0,-3.5
9067,9,New York Giants,0.568567,9.0,New York Giants,6.0,2.5
9066,9,Miami Dolphins,0.565851,8.0,Miami Dolphins,10.0,3.5
9072,9,San Diego Chargers,0.565649,7.0,San Diego Chargers,11.0,5.0


In [17]:
week_filter = dfAll.gameWeek == 7
dfAll['predictTeam'] = np.where((dfAll['predict_proba'] - .5) > 0 , dfAll['favorite'], dfAll['underdog'])
dfAll[week_filter][predictCols].sort(guessCol, ascending=False)

,gameWeek,predictTeam,predict_proba,probaGuess,favorite,lineGuess,Line
8786,7,New England Patriots,0.838849,16,New England Patriots,16,9.0
8790,7,Arizona Cardinals,0.829496,15,Arizona Cardinals,15,7.5
8788,7,New York Giants,0.693856,14,New York Giants,7,3.5
8780,7,Atlanta Falcons,0.689779,13,Atlanta Falcons,9,-4.5
8777,7,Seattle Seahawks,0.675882,12,Seattle Seahawks,14,-6.0
8789,7,Carolina Panthers,0.668644,11,Carolina Panthers,5,3.0
8781,7,Indianapolis Colts,0.651330,10,Indianapolis Colts,11,4.5
8778,7,Buffalo Bills,0.647091,9,Buffalo Bills,12,-5.5
8784,7,St Louis Rams,0.640078,8,St Louis Rams,13,5.5
8782,7,Minnesota Vikings,0.636101,7,Minnesota Vikings,4,-2.5


In [18]:
# display weekly ranking output for spread method

# ranking methods choices
# 0. pick based on spread
# 1. always pick favored team, rank by probability of win
# 2. pick winner based on abs(probability - .5), rank by probability
# 3. pick winner based on abs(probability - .5), rank by abs(probability - .5)

predictCols = ['favorite','lineGuess', 'absLine','Line', 'favoredHomeGame', 'divisionGame', 'favoredRecord']

sortCols = ['absLine','favoredHomeGame', 'divisionGame', 'favoredRecord', 'favorite']
dfSpread = dfAll[predictCols].sort(sortCols , ascending=False)
#print(dfSpread.to_csv(sys.stdout,sep=',', index=False))
dfSpread

,favorite,lineGuess,absLine,Line,favoredHomeGame,divisionGame,favoredRecord
8786,New England Patriots,16,9.0,9.0,1,1,0.833333
8790,Arizona Cardinals,15,7.5,7.5,1,0,0.666667
8777,Seattle Seahawks,14,6.0,-6.0,0,1,0.333333
8784,St Louis Rams,13,5.5,5.5,1,0,0.333333
8778,Buffalo Bills,12,5.5,-5.5,0,0,0.500000
8781,Indianapolis Colts,11,4.5,4.5,1,0,0.500000
8785,Miami Dolphins,10,4.5,4.5,1,0,0.333333
8780,Atlanta Falcons,9,4.5,-4.5,0,0,0.833333
8787,San Diego Chargers,8,4.0,4.0,1,1,0.333333
8788,New York Giants,7,3.5,3.5,1,1,0.500000
